# Code for Producing Figure 1  

### From the paper:  
**"Fractal clusters and urban scaling shape spatial inequality in U.S. patenting"**, published in npj Complexity,  https://doi.org/10.1038/s44260-025-00054-y

**Authors:**  
Salva Duran-Nebreda, Blai Vidiella,  R. Alexander Bentley and Sergi Valverde

**Date:** September 8, 2025  
**Location:** Barcelona  
**Group:** Evolution of Networks Lab |  https://svalver.github.io  
**E-mail:** svalver@gmail.com

In [2]:
import powerlaw
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.spatial.distance import pdist, euclidean
from scipy.optimize import curve_fit

def func_powerlaw(x, m, c):
    return c*x**m

def get_confidence_intervals(popt, pcov, alpha=0.05):
    perr = np.sqrt(np.diag(pcov))
    n_params = len(popt)
    z = 1.96  # 95% confidence interval (alpha=0.05)
    conf_intervals = []
    for i in range(n_params):
        conf_intervals.append((popt[i] - z * perr[i], popt[i] + z * perr[i]))
    return conf_intervals

In [6]:
DATAFOLDER = "../../data/"

In [9]:
for dataset in ['apps2', 'plugin2', 'fracking', 'ml', 'videogames', 'biotech', 'smartphone', 'chips']:
    df = pd.read_csv(DATAFOLDER + 'full_' + dataset + '.csv', encoding="latin-1")

    for i in range(len(df)):
        if df.loc[i, 'country'] == 'USA':
            df.loc[i, 'country'] = 'US'

    for country in ['US']:#, 'JP']:
        city_dict = {}
        df_country = df[df['country'] == country]
        df_country.reset_index(drop=True, inplace=True)

        for i in range(len(df_country)):
            city = (df_country.loc[i, 'longitude'], df_country.loc[i, 'latitude'])
            if city not in city_dict:
                city_dict[city] = 1
            else:
                city_dict[city] += 1

        freq_rank = sorted(city_dict.values(), reverse=True)
        print(dataset, country, len(df_country), len(freq_rank))
        results = powerlaw.Fit(freq_rank, discrete=True)

        plt.figure(figsize=(4.5,4.5))
        string = results.power_law.alpha
        xmin = results.xmin

        x_start = int(len(freq_rank)/100)

        fit_freq = freq_rank[x_start:]
        print(x_start, len(freq_rank), len(fit_freq))
        fit_rank = [x + x_start + 1 for x in range(len(fit_freq))]
        popt, pcov = curve_fit(func_powerlaw, fit_rank, fit_freq, maxfev=10**5)
        ci_power_law = get_confidence_intervals(popt, pcov)
        error = ci_power_law[0][0] - popt[0]

        fig_ccdf = results.plot_ccdf(label="CCDF", color='b', marker='o', linewidth=0)
        results.power_law.plot_ccdf(results.data, color='r', label="Fit", ax=fig_ccdf)
        plt.xlabel("Number of patents")
        plt.ylabel("Frequency")
        plt.title(dataset + ' ' +country+r' $, \alpha=$' + str(round(string-1, 3)) + r'$\pm$' + str(round(abs(error),4))
                  +r' $, x_{min}=$' + str(round(xmin, 3)))#+r' $, x_{max}=$' + str(round(xmax, 3)))
        plt.xscale('log')
        plt.yscale('log')
        plt.legend()
        plt.savefig('./Final/CCDF_' + dataset + ' ' + country + '.png')
        plt.close()

        plt.figure(figsize=(4.5,4.5))
        rank = [x + 1 for x in range(len(freq_rank))]

        fit_freq = freq_rank[x_start:]
        fit_rank = [x + x_start + 1 for x in range(len(fit_freq))]

        popt, pcov = curve_fit(func_powerlaw, fit_rank, fit_freq, maxfev=10**5)
        ci_power_law = get_confidence_intervals(popt, pcov)
        error = ci_power_law[0][0] - popt[0]

        print(country, 'Zipf exponent:', popt[0])
        fig_fr = plt.plot(rank,freq_rank, color='b', marker='o', linewidth=0, label='Frequency Rank')
        plt.plot(rank, func_powerlaw(rank, *popt), color='r', label='Fit')
        plt.ylabel("Number of patents")
        plt.xlabel("Rank")
        plt.title(dataset + ' ' + country + r' $, \alpha=$' + str(round(popt[0], 3)) + r'$\pm$' + str(round(abs(error),4))
                  + r' $, x_{min}=$' + str(round(xmin, 3)))  # +r' $, x_{max}=$' + str(round(xmax, 3)))
        plt.xscale('log')
        plt.yscale('log')
        plt.legend()
        plt.savefig('./Final/Zipf_' + dataset + ' ' + country + '.png')
        plt.close()

        R, p = results.distribution_compare('truncated_power_law', 'exponential')
        print(dataset, country, ' Truncated PL/Exponential Likelihood ratio: ', R, 'p-value: ', p)

        R, p = results.distribution_compare( 'truncated_power_law', 'power_law')
        print(dataset, country, ' Truncated PL/Power-law Likelihood ratio: ', R, 'p-value: ', p)

        R, p = results.distribution_compare('truncated_power_law', 'lognormal')
        print(dataset, country, ' Truncated PL/Log-normal Likelihood ratio: ', R, 'p-value: ', p)
        print()

apps2 US 15637 1356
Calculating best minimal value for power law fit
13 1356 1343s: 98%
US Zipf exponent: -0.9718511197385404
apps2 US  Truncated PL/Exponential Likelihood ratio:  92.93601013877272 p-value:  2.0159384277424004e-06
apps2 US  Truncated PL/Power-law Likelihood ratio:  3.2626144344860366 p-value:  0.010635486248999126
apps2 US  Truncated PL/Log-normal Likelihood ratio:  0.9039620871296075 p-value:  0.15602205442259684

plugin2 US 3527 856
Calculating best minimal value for power law fit
8 856 848ress: 97%
US Zipf exponent: -0.8776595230793157


Assuming nested distributions


plugin2 US  Truncated PL/Exponential Likelihood ratio:  46.96138553552258 p-value:  0.00016108980488463873
plugin2 US  Truncated PL/Power-law Likelihood ratio:  2.0801723670510643 p-value:  0.04138098097231124
plugin2 US  Truncated PL/Log-normal Likelihood ratio:  0.6982750741846067 p-value:  0.12655079700785773

fracking US 3645 810


Assuming nested distributions


Calculating best minimal value for power law fit
8 810 802ress: 97%
US Zipf exponent: -1.0478335076128429
fracking US  Truncated PL/Exponential Likelihood ratio:  116.17704799457567 p-value:  2.179755436171943e-05
fracking US  Truncated PL/Power-law Likelihood ratio:  0.5054229418385849 p-value:  0.3147002767319643
fracking US  Truncated PL/Log-normal Likelihood ratio:  0.3744763053597868 p-value:  0.20091639720777255

ml US 14603 1520


...

Assuming nested distributions


smartphone US  Truncated PL/Exponential Likelihood ratio:  34.13648907997372 p-value:  0.00012444318558181892
smartphone US  Truncated PL/Power-law Likelihood ratio:  1.013385638007565 p-value:  0.15454863275143316
smartphone US  Truncated PL/Log-normal Likelihood ratio:  0.3803937790867753 p-value:  0.17027544417567986

chips US 3045 413
Calculating best minimal value for power law fit
4 413 409ress: 97%
US Zipf exponent: -1.0971356384332032


Assuming nested distributions


chips US  Truncated PL/Exponential Likelihood ratio:  145.32209761963585 p-value:  6.072369924056378e-11
chips US  Truncated PL/Power-law Likelihood ratio:  1.4742971496016395 p-value:  0.0859522909827437
chips US  Truncated PL/Log-normal Likelihood ratio:  1.0171372908569352 p-value:  0.008482131493520053



Assuming nested distributions
